<a href="https://colab.research.google.com/github/michealman114/Natural-Language-Models-for-Hate-Speech-Classification/blob/main/Embeddings_from_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
TODO: Decrease embedding size from BERT, currently is at least 768
TODO: Further improve embedding choice - either: sum last 4 hidden layers OR concatenate last 4 hidden layers
TODO: Switch from DistilBert to Bert?
"""

In [ ]:
!pip install transformers

In [ ]:
from transformers import DistilBertTokenizer, DistilBertModel, DistilBertConfig

import torch
import torch.nn as nn 
import torch.utils.data as torch_data

import numpy as np
import json

In [ ]:
from torch import cuda

if cuda.is_available():
    device = 'cuda'
    seed = 4814
    torch.cuda.manual_seed_all(seed)
    print("running on GPU:", torch.cuda.get_device_name(0))
else:
    device = 'cpu'
    print("running on CPU")

running on GPU: Tesla P100-PCIE-16GB


In [ ]:
def getCommentsTitlesLabels(file_lines):
    comment_list = []
    title_list = []
    labels = []
    for line in file_lines:
        content = json.loads(line)

        comment = content['text']
        comment_list.append(comment)

        title = content['title']
        title_list.append(title)

        labels.append(content['label'])
    
    return comment_list,title_list,labels

def custom_shuffle(comments_embeddings,titles_embeddings,labels):
    """
    comments/title is a (batch_size, max_comment/title_length,embedding size) ndarray, this means we need batch_first=true in nn.LSTM
    comment_array.shape = (batch_size, max_comment_len, 300) #300 is wor2vec embedding size
    labels.shape = (batch_size,)
    """
    num_samples, _ , _ = comments_embeddings.shape
    shuffled_indices = np.random.permutation(num_samples) #return a permutation of the indices
    
    shuffled_comments = comments_embeddings[shuffled_indices,:,:]
    shuffled_titles = titles_embeddings[shuffled_indices,:,:]
    shuffled_labels = labels[shuffled_indices]

    return (shuffled_comments, shuffled_titles, shuffled_labels)

In [ ]:
config = DistilBertConfig.from_pretrained('distilbert-base-uncased', output_hidden_states=True)
model = DistilBertModel.from_pretrained('distilbert-base-uncased', config=config).to(device)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#test_lines = open("./Data/modern_comments.json", "r").readlines() # modern data
#test_comments,test_titles,test_labels = getCommentsTitlesLabels(test_lines)
#test_labels = torch.tensor(test_labels)

#tokenized_test_comments = tokenizer(train_comments, padding = True, truncation = True, return_tensors="pt").to(device)
#tokenized_test_titles = tokenizer(train_titles, padding = True, truncation = True, return_tensors="pt").to(device)

In [ ]:
train_lines = open("./Data/fox-news-comments.json", "r").readlines() # original 2015 data
train_comments, train_titles, train_labels = getCommentsTitlesLabels(train_lines)
train_labels = torch.tensor(train_labels)

In [ ]:
train_labels.shape[0]

1528

In [ ]:
class CommentsDataset(torch.utils.data.Dataset):
    def __init__(self, comments, titles, labels):
        """
        comments/titles: (batch_size, max_length, embed_dim)
        labels: (batch_size,)
        """
        #Initialization
        self.comments = comments
        self.titles = titles
        self.labels = labels
        self.length = labels.shape[0]

    def __len__(self):
        return self.length

    def __getitem__(self, index):
        # Load data and get label
        comment = self.comments[index]
        title = self.titles[index]
        label = self.labels[index]

        return comment,title,label

In [ ]:
train_data = CommentsDataset(train_comments, train_titles, train_labels)
train_loader = torch_data.DataLoader(train_data, batch_size=128, shuffle=True)

In [ ]:
print(train_comments[:5])

['Merkel would never say NO', 'Expect more and more women to be asking .. "why are men no longer interested in me"! We\'re not going touch you until you pull our pants down!', "Groping people in public wasn't already illegal? What's up with that, Deutschland?", 'Merkel, possible the only person in charge who is worse than what we have. Obama is trying his hardest though to get to Merkel\'s level. "A 21-year-old Iraqi man was convicted of sexual assault and given a one-year suspended sentence. A 26-year-old Algerian man was convicted of abetting a sexual assault and attempted assault, and given the same sentence." Sounds exactly like how Obama is trying to let all of the black people in this country get away with everything.', 'They know very well, no means NO! They need to pass a law making it legal to castrate those animals.']


In [ ]:
# Put the model in eval mode to turn off dropout regularization etc.
model.eval()

embedded_train_comments = []
embedded_train_titles = []
embedded_train_labels = []

i = 1

#use torch.no_grad() to speed up the embedding process
with torch.no_grad():
    for comments, titles, labels in train_loader:
        """
        comments: list of size (num_comments)
        titles: list of size (num_comments)
        labels: torch.tensor of size (num_comments)
        """

        tokenized_train_comments = tokenizer(comments, padding = "max_length", max_length = 512, truncation = True, return_tensors="pt").to(device)
        tokenized_train_titles = tokenizer(titles, padding = "max_length", max_length = 512, truncation = True, return_tensors="pt").to(device)


        train_comments_outputs = model(**tokenized_train_comments) # (last_hidden_state,hidden_states[optional], attentions[optional])
        train_titles_outputs = model(**tokenized_train_titles) # (last_hidden_state,hidden_states[optional], attentions[optional])

        all_train_comments_embeddings = train_comments_outputs[1] #tuple of hidden states from each layer of DistilBERT
        all_train_titles_embeddings = train_titles_outputs[1] #tuple of hidden states from each layer of DistilBERT

        final_train_comments_embeddings_temp = all_train_comments_embeddings[-2]
        final_train_titles_embeddings_temp = all_train_titles_embeddings[-2]

        #print(final_train_comments_embeddings_temp.shape) #(batch_size=128, length = 512, embed_dim = 768)

        train_comments.append(final_train_comments_embeddings_temp)
        embedded_train_comments.append(final_train_comments_embeddings_temp)
        embedded_train_titles.append(final_train_titles_embeddings_temp)
        embedded_train_labels.append(labels)

        print(f'iteration {i} completed')
        i+= 1



final_train_comments_embeddings = torch.cat(embedded_train_comments)
final_train_titles_embeddings = torch.cat(embedded_train_titles)

print(final_train_comments_embeddings.shape)
print(final_train_titles_embeddings.shape)


iteration 1 completed
iteration 2 completed
iteration 3 completed
iteration 4 completed
iteration 5 completed
iteration 6 completed
iteration 7 completed
iteration 8 completed
iteration 9 completed
iteration 10 completed
iteration 11 completed
iteration 12 completed
torch.Size([1528, 512, 768])
torch.Size([1528, 512, 768])


In [ ]:

tokenized_train_comments = tokenizer(train_comments, padding = True, truncation = True, return_tensors="pt")
tokenized_train_titles = tokenizer(train_titles, padding = True, truncation = True, return_tensors="pt")



In [ ]:
print(tokenized_train_comments['input_ids'].shape)

torch.Size([1528, 512])


In [ ]:
# Put the model in eval mode to turn off dropout regularization etc.
model.eval()


#use torch.no_grad() to speed up the embedding process
with torch.no_grad():
    train_comments_outputs = model(**tokenized_train_comments) # (last_hidden_state,hidden_states[optional], attentions[optional])
    last_train_comments_embeddings = train_comments_outputs[0]
    all_train_comments_embeddings = train_comments_outputs[1] 
    # comments_embeddings is a tuple of hidden states at each layer of DistilBERT
    # (comments_embeddings[0] corresponds to first layer, comments_embeddings[6] corresponds to last hidden layer)

    train_titles_outputs = model(**tokenized_train_titles) # (last_hidden_state,hidden_states[optional], attentions[optional])
    last_train_titles_embeddings = train_titles_outputs[0]
    all_train_titles_embeddings = train_titles_outputs[1] 


    """
    test_comments_outputs = model(**tokenized_test_comments) # (last_hidden_state,hidden_states[optional], attentions[optional])
    last_test_comments_embeddings = test_comments_outputs[0]
    all_test_comments_embeddings = test_comments_outputs[1]

    test_titles_outputs = model(**tokenized_test_titles) # (last_hidden_state,hidden_states[optional], attentions[optional])
    last_test_titles_embeddings = test_titles_outputs[0]
    all_test_titles_embeddings = test_titles_outputs[1] 
    """

In [ ]:
print(last_train_comments_embeddings[:1])
print(all_train_comments_embeddings[6][:1])

In [ ]:
#TODO: produce best embeddings by summing the last 4 layers (for DistilBert vs Bert this would probably be smaller)
#Ref: http://jalammar.github.io/illustrated-bert/ https://medium.com/@dhartidhami/understanding-bert-word-embeddings-7dc4d2ea54ca

final_train_comments_embeddings = all_train_comments_embeddings[5]
final_train_titles_embeddings = all_train_titles_embeddings[5]
print(final_train_comments_embeddings.shape)
print(final_train_titles_embeddings.shape)

torch.Size([5, 106, 768])
torch.Size([5, 16, 768])


In [ ]:
final_train_comments_embeddings, final_train_titles_embeddings, train_labels = custom_shuffle(final_train_comments_embeddings, final_train_titles_embeddings, train_labels)